In [1]:
import sys,os
pathThisFile=sys.path[0]
print(pathThisFile)

C:\Users\guendm1\Documents\BerlinDenkmal\Denkmal-Berlin\Vorgang


In [2]:
import json

In [5]:
import Vorgang.getVorgang as gVorgang
findVorgang=Vorgang.findVorgang

ModuleNotFoundError: No module named 'Vorgang'

In [4]:
#target_dir = r"C:\Users\schull\Projekte\KIbarDok\testDatenMerge"
#target_dir = r"C:\Users\guendm1\Documents\BerlinDenkmal\Testdaten\All\doc"
#target_dir = r"C:\Users\guendm1\Documents\BerlinDenkmal\Testdaten\AllSepType\doc"
target_dir = r"C:\Users\guendm1\Documents\BerlinDenkmal\Testdaten\Sep\Wilhelminenhofstr76-78"
os.chdir(target_dir)
files = sorted(os.listdir(target_dir))

In [5]:
def rep(x):
    return str(x).replace('{','').replace('}','').replace('[','').replace(']','').replace('\"','').replace('\'','')

In [6]:
def updatedResult(oldResult,newResult):
    for key in oldResult['Ergebnis']:
        oldResult['Ergebnis'][key]=oldResult['Ergebnis'][key]+newResult['Ergebnis'][key]
    for filePath in newResult['Gesamt-Zuordnung'].keys():
        oldResult['Gesamt-Zuordnung'].setdefault(filePath,{}).update(newResult['Gesamt-Zuordnung'][filePath])
        oldResult['Keine-Zuordnung'].setdefault(filePath,{}).update(newResult['Keine-Zuordnung'][filePath])
        oldResult['Informelles-Format'].setdefault(filePath,[]).extend(newResult['Informelles-Format'][filePath])
        oldResult['Fehler'].setdefault(filePath,[]).extend(newResult['Fehler'][filePath])
    return oldResult

In [7]:
def save(filePath,considerDocName,newResult):
    if considerDocName==True:
        name=pathThisFile+'\outputResult\\'+'result-docName.json'            
    else:
        name=pathThisFile+'\outputResult\\'+'result-NodocName.json'    
    ###########################################
    if not os.path.isfile(name):#create json-result file
        with open(name,'w') as fp:
            json.dump(newResult, fp,  indent=4,ensure_ascii=False)
            print('resultFile0',newResult)
    else:#update the json-result file
        with open(name,'r') as fp:
            resultFile = json.load(fp)
        #print('resultFile',resultFile)
        #print('newResult', newResult)
        finalResult=updatedResult(resultFile,newResult)
        #print('finalResult',finalResult)
        with open(name,'w') as f:
            json.dump(finalResult, f,  indent=4,ensure_ascii=False)

In [8]:
def allVorgang(file,filePath,considerDocName,methode):
    resultAll=findVorgang(file, filePath, considerDocName,methode).all
    save(filePath,considerDocName,resultAll)
    return resultAll

In [9]:
def vorgang(files,filePath,considerDocName,methode):
    if type(files)!=list:
        files=[files]
    alle=allVorgang(files,filePath,considerDocName,methode)
    zuordnung=alle['Gesamt-Zuordnung']
    keineZuordnung=alle['Keine-Zuordnung']
    informellesFormat=alle['Informelles-Format'] 
    fehler=alle['Fehler']
    ###############################################
    result={}
    result[filePath]={}
    ###############################################
    for f in files:
        result[filePath][f]={}
        if f in zuordnung[filePath].keys():
            vResult=rep(list(zuordnung[filePath][f].keys()))
        elif f in keineZuordnung[filePath].keys():
            vResult='Keine Kategorie gefunden'
        elif f in informellesFormat[filePath]:
            vResult='Informelles Format'
        elif f in fehler[filePath]:
            vResult='Fehler in der Datei'
        else:
            vResult='Fehler in der Datei'
        result[filePath][f]['vorgang']=vResult
    return result

In [ ]:
considerDocName=True
finalResult={}
i=0
for root, dirs, files in os.walk(target_dir):
    for file in files:
        finalResult[root]={}
        finalResult[root][file]=vorgang(file, root, considerDocName,'tika')
        print(finalResult[root][file])

pages:  59
File is too long for Behördendokument: 00_Auslobung_htw-fki_lo.pdf
{'C:\\Users\\guendm1\\Documents\\BerlinDenkmal\\Testdaten\\Sep\\Wilhelminenhofstr76-78': {'00_Auslobung_htw-fki_lo.pdf': {'Vorgang': 'Informelles Format'}}}
pages:  2
{'C:\\Users\\guendm1\\Documents\\BerlinDenkmal\\Testdaten\\Sep\\Wilhelminenhofstr76-78': {'070927 Vermerk Abstimmung Fassade .pdf': {'Vorgang': 'Informelles Format'}}}
pages:  1
{'C:\\Users\\guendm1\\Documents\\BerlinDenkmal\\Testdaten\\Sep\\Wilhelminenhofstr76-78': {"152-13,W'hofstr.74-76_Errichtung Werbung.doc": {'Vorgang': 'Antrag'}}}
pages:  13
{'C:\\Users\\guendm1\\Documents\\BerlinDenkmal\\Testdaten\\Sep\\Wilhelminenhofstr76-78': {'70618 FHTW 1.BA -Hauptmaßnahme- Projektbeteiligte 070124.pdf': {'Vorgang': 'Informelles Format'}}}
pages:  1
{'C:\\Users\\guendm1\\Documents\\BerlinDenkmal\\Testdaten\\Sep\\Wilhelminenhofstr76-78': {'Abgabenachr.an Ord L_Lärmbelästig..doc': {'Vorgang': 'Keine Kategorie gefunden'}}}
{'C:\\Users\\guendm1\\Document

In [ ]:
print(finalResult)